In [1]:
# !export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems
from src.embeddings import RecsysEmbedding

experiment_name = 'SessionwiseAttention'
device = 'cuda:0'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
class SessionwiseAttention(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention= torch.nn.MultiheadAttention(
            self.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(2 * embedding.embedding_dim, output_dim)

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        device = item_embs.device
        
        # flattening slates in to long sequences
        item_embs = item_embs.flatten(1, 2)
        
        # let model attent to first padd token if slate is empty to avoid NaN gradients
        # (anyway they does not contrinute into metrics computation)
        key_padding_mask = batch['slates_mask'].clone()
        key_padding_mask[:,:, 0] = True # let model attent to first padd token if slate is empty 
        key_padding_mask = ~key_padding_mask.flatten(1,2)
        
        # forbid model looking into future (and into current iteraction)
        future_mask = torch.ones((item_embs.size(-2), item_embs.size(-2))).to(device)
        future_mask = torch.triu(future_mask, diagonal=1).bool()
        
        features, attn_map = self.attention(
            item_embs, item_embs, item_embs,
            key_padding_mask=key_padding_mask,
            attn_mask = future_mask
        )
#         print(features.shape, user_embs.shape, shp)
        features = torch.cat(
            [
                features.reshape(shp),
                user_embs[:, :, None, :].repeat(1, 1, shp[-2], 1)
            ],
            dim = -1
        )
        
        return self.out_layer(features).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SessionwiseAttention(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device='cpu', lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3956.89it/s]


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.6666666269302368, 'accuracy': 0.25}


train... loss:0.6950739622116089:   0%|                                                                                                    | 1/5000 [00:00<26:50,  3.10it/s]

Val update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.52
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 


train... loss:0.6950739622116089:   0%|                                                                                                    | 1/5000 [00:00<51:44,  1.61it/s]

Val update: epoch: 1 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.5800000000000001
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SessionwiseAttention(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (attention): MultiheadAttention(
     (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
   )
   (out_layer): Linear(in_features=64, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))

# another batch size due to large memory usage
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=10)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 1618 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SessionwiseAttention(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SessionwiseAttention with svd embeddings
Test before learning: {'f1': 0.06840086728334427, 'roc-auc': 0.5205211639404297, 'accuracy': 0.8633807301521301}


train... loss:522.6785174459219:   0%|                                                                                                | 1/5000 [02:23<199:10:45, 143.44s/it]

Val update: epoch: 0 |accuracy: 0.8171793818473816 | f1: 0.23787805438041687 | auc: 0.6180979013442993 | treshold: 0.09
Test: accuracy: 0.8175572752952576 | f1: 0.24274539947509766 | auc: 0.6207095384597778 | 


train... loss:497.3637044876814:   0%|                                                                                                | 2/5000 [04:14<172:28:49, 124.24s/it]

Val update: epoch: 1 |accuracy: 0.8029280304908752 | f1: 0.23885244131088257 | auc: 0.6211483478546143 | treshold: 0.13
Test: accuracy: 0.7982856035232544 | f1: 0.24761904776096344 | auc: 0.625458300113678 | 


...

train... loss:480.38318118453026:   0%|                                                                                               | 2/5000 [03:46<157:12:13, 113.23s/it]

Val update: epoch: 1 |accuracy: 0.8199145197868347 | f1: 0.2667057514190674 | auc: 0.6933383345603943 | treshold: 0.13
Test: accuracy: 0.8221928477287292 | f1: 0.27672332525253296 | auc: 0.6985437273979187 | 


train... loss:467.45993869006634:   0%|                                                                                               | 6/5000 [11:01<152:50:35, 110.18s/it]

Val update: epoch: 5 |accuracy: 0.7718197107315063 | f1: 0.27874594926834106 | auc: 0.6984010934829712 | treshold: 0.14
Test: accuracy: 0.7737552523612976 | f1: 0.2841204106807709 | auc: 0.7038165926933289 | 


train... loss:462.80683448910713:   0%|▍                                                                                             | 21/5000 [39:24<155:43:53, 112.60s/it]


In [14]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

NameError: name 'dataset' is not defined

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SessionwiseAttention(
        RecsysEmbedding(
            train_num_items,
            train_user_item_matrix,
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SessionwiseAttention with explicit embeddings
Test before learning: {'f1': 0.5757297277450562, 'roc-auc': 0.5344966650009155, 'accuracy': 0.5195502638816833}


train... loss:60.10406452417374:   0%|                                                                                                  | 1/5000 [00:59<82:48:07, 59.63s/it]

Val update: epoch: 0 |accuracy: 0.7888550162315369 | f1: 0.8451688289642334 | auc: 0.8453947901725769 | treshold: 0.46
Test: accuracy: 0.7957683205604553 | f1: 0.8509669899940491 | auc: 0.8532283306121826 | 


train... loss:53.102506309747696:   0%|                                                                                                 | 2/5000 [01:05<39:13:49, 28.26s/it]

Val update: epoch: 1 |accuracy: 0.7928215265274048 | f1: 0.8477967381477356 | auc: 0.8560933470726013 | treshold: 0.43
Test: accuracy: 0.8009672164916992 | f1: 0.8546350598335266 | auc: 0.8639253973960876 | 


...

train... loss:49.54580545425415:   1%|█▏                                                                                                | 63/5000 [06:27<8:18:50,  6.06s/it]

Val update: epoch: 62 |accuracy: 0.8031490445137024 | f1: 0.8478549122810364 | auc: 0.8747761249542236 | treshold: 0.45
Test: accuracy: 0.8118002414703369 | f1: 0.855267345905304 | auc: 0.8807227611541748 | 


train... loss:49.58063179254532:   1%|█▎                                                                                                | 65/5000 [06:39<8:22:56,  6.11s/it]

Val update: epoch: 64 |accuracy: 0.7774149775505066 | f1: 0.8452106714248657 | auc: 0.8749202489852905 | treshold: 0.42000000000000004
Test: accuracy: 0.7818643450737 | f1: 0.8486688733100891 | auc: 0.8809521794319153 | 


train... loss:49.46283343434334:   1%|█▎                                                                                                | 66/5000 [06:51<8:32:53,  6.24s/it]


In [12]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items